In [10]:
import pandas as pd
import MeCab
import sys

mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

In [3]:
stop_words = ['円','点','時','分','!','・',';','?','年','月','日','゚ Д ゚','ウマー']

In [4]:
review_df = pd.read_csv('../csv/review.csv')

In [5]:
review_df.head()

,store_id,store_name,score,station,review
0,1,中華そば しば田,4.06,仙川駅,開店前から長蛇の列!!( ; ﾛ)ﾟ ﾟ開店前10分前に到着しましたが、ありつけるまで、1時...
1,2,Homemade Ramen 麦苗,4.02,大森海岸駅,わがままボディーの後輩との東京遠征2日目の昼。実に4年ぶりとなるこちらへ。当時から行列のでき...
2,3,手打式超多加水麺 ののくら,3.98,亀有駅,2020・1・28 初訪TRY本で新人賞を獲った頃から行きたかった「ののくら」。今日の午前中...
3,4,中華そば 多賀野,3.97,荏原中延駅,中華そば750、大盛120動物系乾物系背脂醤油スープ中細ストレート麺ちょいカタメ豚バラチャー...
4,5,麺庵ちとせ,3.97,若松河田駅,百名店ラーメンTOKYO2019の51店目は曙橋の『麺庵ちとせ』さんへ！地下鉄の駅からお店へ...


In [6]:
review_df['station'] = review_df['station'].str.rstrip('駅')

In [7]:
def remove_stop_words(x):
    for s in stop_words:
        x = x.replace(s, '')
    return x

In [8]:
review_df["review"] = review_df["review"].apply(remove_stop_words)

In [9]:
review_df['review'] = review_df['review'].str.normalize('NFKC')

In [23]:
def get_noun(text):
    mecab.parse('')
    node = mecab.parseToNode(text)
    result = []

    while node:
        word = node.surface
        pos,pos2 = node.feature.split(",")[0],node.feature.split(",")[1]
        if '0' in word or '1' in word or '2' in word or '3' in word or '4' in word or '5' in word or '6' in word or '7' in word or '8' in word or '9' in word:
            node = node.next
            continue

        if pos2 == '数':
            node = node.next
            continue
            
        if (pos == '名詞' and pos2 != '数') or (pos == '形容詞') or (pos == '副詞'):
            result.append(word)
        node = node.next

    return ' '.join(result)

In [24]:
review_df['wakati_review'] = review_df['review'].apply(get_noun)

In [25]:
review_df.to_csv('../csv/review_wakati.csv')